In [1]:
!pip install -q tritonclient[http]
!pip install -q pandas scikit-learn

In [2]:
import tritonclient.http as httpclient
from tritonclient.utils import np_to_triton_dtype

In [3]:
import pandas as pd
import numpy as np
import json

In [4]:
client = httpclient.InferenceServerClient(url='localhost:8000')

In [5]:
data_infer = pd.read_csv("data_infer.csv").astype('str')

In [6]:
data_infer.head()

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?
0,1005,0,2013,6,5,22:03,$-180.00,Swipe Transaction,7834055923142137930,Orlando,FL,32808.0,3395,nan
1,134,5,2015,2,16,18:34,$99.10,Chip Transaction,-1548923525906069124,Tyler,TX,75706.0,4900,nan
2,1270,0,2009,11,27,18:44,$82.42,Swipe Transaction,-7329090674102508598,Saint Charles,IL,60174.0,5812,nan
3,109,0,2010,11,3,16:21,$0.45,Swipe Transaction,-6571010470072147219,Dyer,IN,46311.0,5499,nan
4,1853,2,2006,3,27,12:25,$11.41,Swipe Transaction,6135208568923449408,East Peoria,IL,61611.0,9402,nan


In [17]:
input_data = data_infer.iloc[:4].values

In [18]:
def prepare_tensor(name, input_data):
    tensor = httpclient.InferInput(
        name, input_data.shape, np_to_triton_dtype(input_data.dtype))
    tensor.set_data_from_numpy(input_data)
    return tensor

In [24]:
model_name = "ensemble_preprocess_fil_postprocess"

In [25]:
inputs = [prepare_tensor('raw_data', input_data)]

In [26]:
response = client.infer(model_name,
                        inputs)

In [27]:
output_data = response.as_numpy("processed_predictions")

In [28]:
output_data.astype(str)

array(['NOT FRAUD', 'NOT FRAUD', 'NOT FRAUD', 'NOT FRAUD'], dtype='<U9')